# ふらっとPythonの会 

#### 2022 / 1 / 24 （ 3 / 7 追記）


## データ分析実践

実際のデータ分析作業には、都度外部モジュールを使うことになります。

ここでは、西粟倉村のオープンデータをダウンロードし、グラフにしてみましょう。

作業には、以下のステップを踏みます。

- インターネットからデータのダウンロード
- データの成形
- 可視化

## 2_1. データ読み込み

インターネットへのアクセスには `requests` を使用します。

おかやまオープンデータカタログ：
https://www.okayama-opendata.jp/

In [ ]:
import requests

url='http://vill.nishiawakura.okayama.jp/open_data/jinko/nenreibetsu/202104_nishiawakura.xls'

filename='R3人口データ.xls'
urlData = requests.get(url).content

with open(filename ,mode='wb') as f:
    f.write(urlData)

これで、ダウンロードが完了し、現在のディレクトリに保存されたことになります。

In [ ]:
# 以下のようにして、現在のディレクトリを明示することもできます。
import os
os.getcwd()

## 2_2. 前処理

前処理とは、データ分析作業の前段階で、データの成形や型の調整、外れ値やデータ以上の確認など、分析時に意図しない結果を返さないように、データを編集する作業です。

データ分析業務において、実際の分析より時間がかかる場合が多く、「実作業の9割は前処理に費やす」ともいわれます。

今回は、Pythonでデータ分析を行う上で最もよく使うモジュールのひとつ`Pandas`を、前処理部分で使います。

`Pandas`は`Numpy`と並んで最頻のモジュールであり、強力なデータ構造である`Series`と`DataFrame`が提供されています。

エクセルファイルを読み込む専用のメソッド`read_excel`を使用すると、いきなり`DataFrame`型で読み込みが完了します。

In [ ]:
import pandas as pd
df = pd.read_excel("./R3人口データ.xls", sheet_name = "外国人含む")
df

In [ ]:
# .head()や .tail() メソッドで表示範囲を限定することができます。
df.head(10) # <= デフォルトは5

`read_excel`は非常に便利ですが、エクセルファイルを読み込んでも、通常そのまま分析に使える状態にはなりません。

元データによっては、A1セルからデータが始まっていない、ヘッダーが複数行になっている、注記など不要な記述をそのまま読み込んでしまうなどが要因となります。



今回はデータも少ないので、エクセルファイルを直接見ながら、読み込む範囲を変えることで対応します。

In [ ]:
df_jinkou = df.iloc[6:117,1:5]
df_jinkou

#### 3/7追記

`DataFrame.iloc[:,:]`は、DataFrame型のデータの一部を切り取って参照します。

In [ ]:
#体裁をさらに整えます。
df_jinkou.columns = ["age", "male", "female", "total"]
df_jinkou = df_jinkou.reset_index(drop=True)

#### 3/7追記

`DataFrame.columns` に配列を入れると、`DataFrame` の各列を直接書き換えることができます。

`DataFrame.columns` の配列の長さは、もとの `DataFrame` の長さと合わせる必要があります。

また、`DataFrame["列名"]`とすることで、列方向にデータを取り出すことができます。

In [ ]:
# 異常なデータを直接書き換えます。
df_jinkou.iloc[-1, 0] = 109

In [ ]:
df_jinkou

これだけでだいぶスッキリしました。

#### 3/7追記

前処理が完了したデータを詳しく見てみましょう。

In [ ]:
### DataFrame["列名"]を入力し、データを取り出してみましょう。
df_jinkou["age"]

In [ ]:
### DataFrame["列名"]を複数指定するときは、"列名"の部分は配列で指定します。
df_jinkou[["age", "total"]]

In [ ]:
# 条件を指定して、条件に合う特定のデータを取り出す（検索）することもできます。
df_jinkou[df_jinkou["age"] == 60]

## 2_3. 可視化

可視化とは、相手に見せることを目的として、データを視覚的に表現することです。

可視化するときに最も意識しなければならないポイントは、

- 伝えたいことを盛り込むほど、情報量は多くなり、メッセージ性は低くなる。
- 伝えたいことを絞るほど、情報量は少なくなり、メッセージ性は高くなる。

のバランスをうまく取ることです。

In [ ]:
df_jinkou.head()

`Pandas`にはグラフ化のためのメソッドが用意されているので、この状態でも可視化することができます。

In [ ]:
df_jinkou["total"].plot()

データの分布や特徴を見るだけであればこれでも十分でしょう。

グラフにせずに、集計したいのであれば、こちらの方が便利です。

- `pandas.DataFrame().describe()`
- `pandas.DataFrame().info()`
- `pandas.DataFrame().mean()`

In [ ]:
df_jinkou.describe()

今回は、年齢別人口の分布を示すグラフを作ってみます。

先のグラフをそのまま相手に見せるのも良いですが、x軸が詰まって見づらいので、５歳ずつの集計でも問題ないでしょう。

そのための追加の前処理を行います。

In [ ]:
cnt = 0
groupstr = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
grouplist = []
for i in df_jinkou['age']:
    cnt += 1
    grouplist.append(groupstr[i // 5])
    
df_jinkou["group"] = grouplist

#### 3/7追記

「5歳ずつ集計」するにあたり、「どのデータとどのデータを集計するのか」指定するための新しい列を追加しました。

具体的には、「0歳から4歳まで」、「5歳から9歳まで」、・・・、「105歳から109歳まで」とし、

合算されるデータ同士に同じアルファベットを割り振りました。

In [ ]:
df_jinkou

#### 3/7追記

集計する関数は、`DataFrame.groupby()`です。1行で完了します。

In [ ]:
df_group = df_jinkou.groupby(by="group").sum()[["male", "female", "total"]]
df_group = df_group.reset_index(drop=True)

In [ ]:
df_group["bin"] = ["0-4", "5-9", "10-14", "15-19",
                    "20-24", "25-29", "30-34", "35-39",
                    "40-44", "45-49", "50-54", "55-59",
                   "60-64", "65-69", "70-74", "75-79",
                   "80-84", "85-89", "90-94", "95-99",
                   "100-104", "105-109"]

５歳ごとに集計した新しい`DataFrame`：df_groupを作成しました。

In [ ]:
df_group.head()

それではグラフ化してみます。

今回は、使いやすさと見た目の観点から、`matplotlib`を使います。

公式：https://matplotlib.org/


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(16, 8))

#データを指定するオプション
plt.bar(x=df_group["bin"], height=df_group["male"], label='male', align="edge", width= 0.3)
plt.bar(x=df_group["bin"], height=df_group["female"], label='female', align = 'edge', width=-0.3)

#体裁を整えるためのオプション
plt.title("Population in Nishiawakura")
plt.xticks(rotation = 45)
plt.grid(axis='y')
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))

plt.show()

#### 3/7追記

`Matplotlib`には、グラフの描画に必要なオプションが数多く用意されています。

都度、公式Webサイトをみながらグラフを描いてみましょう。

In [ ]:
# プレゼン資料等に使用するため、ディレクトリに保存します。
fig.savefig("img.png")

【注意】

ちなみに、`matplotlib`はデフォルトで日本語の表示に対応していません。

初期状態で日本語を表示させようとすると、`□`で表示されます。（全員こうなります）

日本語にするには、日本語のフォントデータを、インストールフォルダにコピーする必要があります。

結構複雑な上に、環境によって正しいやり方が異なるので、時間があるときに挑戦してみてください。

WindowsでAnacondaを使ってインストールした場合には、調べたかぎり、以下のURLが一番正解に近いですが、Anacondaをインストールした場所によって、コピー先が若干異なります。

http://blog.livedoor.jp/oyajieng_memo/archives/2682215.html

フォントデータは、`IPAexゴシック`が簡単です。https://moji.or.jp/ipafont/ipaex00401/